In [1]:
import pandas as pd 
import numpy as np 
import matplotlib.pyplot as plt
import tensorflow as tf

In [2]:
path_to_file='D:\programming\python\pyTrain\\tensorflow\FINAL_TF2_FILES\TF_2_Notebooks_and_Data\\06-NLP-and-Text-Data\shakespeare.txt'

In [3]:
text=open(path_to_file,'r').read()

In [4]:
print(text[4500:6000])

converted are
  From his low tract and look another way:
    So thou, thy self out-going in thy noon:
    Unlooked on diest unless thou get a son.


                     8
  Music to hear, why hear'st thou music sadly?
  Sweets with sweets war not, joy delights in joy:
  Why lov'st thou that which thou receiv'st not gladly,
  Or else receiv'st with pleasure thine annoy?
  If the true concord of well-tuned sounds,
  By unions married do offend thine ear,
  They do but sweetly chide thee, who confounds
  In singleness the parts that thou shouldst bear:  
  Mark how one string sweet husband to another,
  Strikes each in each by mutual ordering;
  Resembling sire, and child, and happy mother,
  Who all in one, one pleasing note do sing:
    Whose speechless song being many, seeming one,
    Sings this to thee, 'Thou single wilt prove none'.


                     9
  Is it for fear to wet a widow's eye,
  That thou consum'st thy self in single life?
  Ah, if thou issueless shalt hap to die

In [5]:
vocab=sorted(set(text))

In [6]:
len(vocab)

84

In [7]:
char_to_ind={char:ind for ind,char in enumerate(vocab)}
char_to_ind['H']

33

In [8]:
ind_to_char=np.array(vocab)

In [9]:
encoded_text=np.array([char_to_ind[c] for c in text])

In [10]:
encoded_text[43:78]

array([75, 76, 73, 60, 74,  1, 78, 60,  1, 59, 60, 74, 64, 73, 60,  1, 64,
       69, 58, 73, 60, 56, 74, 60,  8,  0,  1,  1, 45, 63, 56, 75,  1, 75,
       63])

In [11]:
encoded_text.shape

(5445609,)

In [12]:
seq_len=120

In [13]:
total_num_seq = len(text)//(seq_len + 1)

In [14]:
total_num_seq

45005

In [15]:
char_dataset=tf.data.Dataset.from_tensor_slices(encoded_text)

In [16]:
type(char_dataset)

tensorflow.python.data.ops.dataset_ops.TensorSliceDataset

In [17]:
sequences=char_dataset.batch(seq_len+1,drop_remainder=True)

In [18]:
def create_seq_target(seq):
    input_text=seq[:-1]
    target_text=seq[1:]
    return input_text,target_text

In [19]:
dataset=sequences.map(create_seq_target)
dataset

<MapDataset shapes: ((120,), (120,)), types: (tf.int32, tf.int32)>

In [20]:
for input_txt,target_txt in dataset.take(1):
    print(input_txt.numpy())
    print("".join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    print("".join(ind_to_char[target_txt.numpy()]))

[ 0  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0
  1  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74
  1 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45
 63 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74
 60  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75]

                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But


[ 1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1  1 12  0  1
  1 31 73 70 68  1 61 56 64 73 60 74 75  1 58 73 60 56 75 76 73 60 74  1
 78 60  1 59 60 74 64 73 60  1 64 69 58 73 60 56 74 60  8  0  1  1 45 63
 56 75  1 75 63 60 73 60 57 80  1 57 60 56 76 75 80  5 74  1 73 70 74 60
  1 68 64 62 63 75  1 69 60 77 60 73  1 59 64 60  8  0  1  1 27 76 75  1]
                     1
  From fairest creatures we desire increase,
  That thereby beauty's rose might never die,
  But 


In [21]:
batch_size=128

In [22]:
buffer_size=10000
dataset=dataset.shuffle(buffer_size).batch(batch_size,drop_remainder=True)

In [23]:
dataset

<BatchDataset shapes: ((128, 120), (128, 120)), types: (tf.int32, tf.int32)>

In [24]:
vocab_size=len(vocab)

In [25]:
embed_dim=64

In [26]:
rnn_neurons=1026

In [27]:
from tensorflow.keras.losses import sparse_categorical_crossentropy

In [28]:
def sparse_cat_loss (y_true,y_pred):
    return sparse_categorical_crossentropy(y_true,y_pred,from_logits=True)

In [29]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding,GRU,Dense

In [30]:
def create_model(vocab_size,embed_dim,rnn_neurons,batch_size):
    model=Sequential()
    model.add(Embedding(vocab_size,embed_dim,batch_input_shape=[batch_size,None]))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    model.add(Dense(vocab_size))
    model.compile('adam',loss=sparse_cat_loss)
    return model

In [31]:
model=create_model(vocab_size=vocab_size,embed_dim=embed_dim,rnn_neurons=rnn_neurons,batch_size=batch_size)

In [32]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (128, None, 64)           5376      
_________________________________________________________________
gru (GRU)                    (128, None, 1026)         3361176   
_________________________________________________________________
dense (Dense)                (128, None, 84)           86268     
Total params: 3,452,820
Trainable params: 3,452,820
Non-trainable params: 0
_________________________________________________________________


In [33]:
"""This notebook ends here, because I dont have a GPU .However i Finished it on google collab """

'This notebook ends here, because I dont have a GPU .However i Finished it on google collab '